<a href="https://colab.research.google.com/github/2SEHI/K-Digital_Lecture/blob/main/0820_pyTorch(LSTM%2C%20GRU%2C%20BERT)/pyTorch(RNN_LSTM%2C_GRU_%EB%8B%A4%EC%9D%8C_%EB%AC%B8%EC%9E%A5_%EC%98%88%EC%B8%A1)ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN - LSTM, GRU, Attention

Transformer : 자연어 처리나 전이 학습


## 1. GRU(Gated Recurrent Unit)란?
- LSTM보다 간단한 구조를 가지고 LSTM과 유사한 성능을 만든 RNN의 변형입니다
- 셀 상태가 별도로 구현하지 않고 셀 상태와 은닉 상태를 하나로 합쳐서 구현하는 모델입니다.

- 망각 게이트(Reset gate)와 업데이트 게이트(Update gate)의 개념을 이용



## 2. LSTM(Long - Term Memory)이란?
- 오랫동안 기억을 하기 위해서 Cell State를 추가한 것
- 기본적인 RNN(현재의 입력 값과 이전 시간의 은닉층 값의 조합으로 새로운 값을 만들어 냄)에 비해 하나의 상태가 더 생긴 것으로 LSTM은 입력 값과 이전 시간의 은닉층 값과 셀 상태의 조합으로 새로운 값을 만들어 냄

- LSTM에서 아래 과정을 거치면 기존의 RNN보다 더 좋은 성능을 가집니다.
    - 셀 상태는 장기기억부분으로 기존의 정보을 얼마나 남겨놓을지 
    - 망각 게이트 : 시그모이드 함수를 적용합니다. 셀 상태 값을 얼마나 잊어버릴 것인지 
    - 입력 게이트 : 시그모이드 함수를 적용. 하이퍼볼릭 탄젠트를 통과시켜서 추가합니다.
    - 은닉 상태 업데이터
    - 망각 게이트(Forget gate)와 입력 게이트(input gate)의 개념을 이용





# 문장 생성 구현을 RNN, GRU, LSTM으로 구현
- 입력받은 문장의 다음 문장을 예측하는 기능을 구현



## 공통 처리

### 1) 사용할 데이터
- 아래 코드를 실행하여 셰익스피어 소설 데이터를 가져옵니다.


In [ ]:
!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt

--2021-08-20 00:17:56--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2021-08-20 00:17:56 (20.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



### 2) 필요한 라이브러리 설치

In [ ]:
!pip install unidecode

     |████████████████████████████████| 241 kB 4.9 MB/s 


### 3) 필요한 라이브러리 import

In [ ]:
import torch
import torch.nn as nn
# 문자열 처리 관련 라이브러리
import string
import re
import unidecode
# 랜덤 모듈
import random
# 시간, 수학 모듈
import time, math

### 4) 하이퍼 파라미터 설정
- 보통 chunk라는 단어가 들어가면 많은 양의 데이터를 분할해서 읽는다는 의미입니다

In [ ]:
num_epochs = 2000
print_every = 100
plot_every = 10

# 많은 양의 데이터 분할
chunk_len = 200

hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

### 5) import 했던 string에서 출력가능한 문자들을 다 불러옴

In [ ]:
# import 했던 string에서 출력가능한 문자들을 다 불러옴
all_characters = string.printable

# 출력가능한 문자들의 개수를 저장함
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 6) 샘플 데이터 읽어오기


In [ ]:
file = unidecode.unidecode(open('/content/input.txt').read())
file_len = len(file)
print(file_len)

1115394


### 7) 샘플을 랜덤하게 불러오는 함수 구현

In [ ]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    # print(start_index)
    return file[start_index:end_index]

print(random_chunk())

enhead, at twelve year old,
I bade her come. What, lamb! what, ladybird!
God forbid! Where's this girl? What, Juliet!

JULIET:
How now! who calls?

Nurse:
Your mother.

JULIET:
Madam, I am here.
What i


### 8) 문자열을 숫자의 list로 변경해주는 함수

In [ ]:
def char_tensor(string):
    # string길이 만큼 0으로 채워진 배열 생성 
    tensor = torch.zeros(len(string)).long()
    
    for c in range(len(string)):
        # string의 문자에 대해 all_characters와 매칭되는 인덱스 번호를 tensor배열에 저장
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor("randomstring"))

tensor([27, 10, 23, 13, 24, 22, 28, 29, 27, 18, 23, 16])


### 9) 랜덤한 텍스트를 가져와서 입력과 목표 값으로 변환해주는 함수
- 다음 글자를 예측처리를 위해 입력할 땐 마지막 글자를 제외하고 
- 출력할 땐 첫글자를 제외합니다

In [ ]:
def random_training_set():
    # 샘플 문장 가져오기
    chunk = random_chunk()
    # print(chunk)
    # print(chunk[:-1])
    # 샘플 문장을 숫자 리스트로 변환

    inp = char_tensor(chunk[:-1])

    target = char_tensor(chunk[1:])
    return inp, target

## Basic RNN 구현

- RNN 모델의 기본 틀

    ```python
    class RNN(nn.Module) : 
        def __init__(super):

        def forward(self, input, hidden):
    ```

### 1) RNN 모델 설계



In [ ]:
class RNN(nn.Module) : 

    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        # 인스턴스 변수 생성
        # 입력의 크기 - 임의 설정 불가능
        self.input_size = input_size 
        # 임베딩사이즈의 크기 - 임의로 정하는 것이 가능
        self.embedding_size = embedding_size 
        # 은닉층에서의 뉴런의 개수 - 임의로 정하는 것이 가능
        self.hidden_size = hidden_size
        # 출력의 개수 - 입력의 크기와 같다
        self.output_size = output_size
        # 은닉층의 개수 - 임의로 지정이 가능
        self.num_layers = num_layers

        # 입력, 은닉, 출력층을 생성

        # 입력층
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        # 은닉층 - 위에 출력(self.embedding_size)을 입력으로 받음
        self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)
        # 출력층
        self.decoder = nn.Linear(self.hidden_size, self.output_size)

    # 순방향 훈련할 때 호출되는 함수
    # nn.Module의 메소드를 overriding
    # RNN과 GRU는 입력과 hidden상태의 조합으로 출력을 생성
    def forward(self, input, hidden):
        
        # 입력층을 사용하는 부분
        # view가 Flatten역할
        out = self.encoder(input.view(1, -1))

        # 은닉층에서는 입력과 이전 가중치를 사용해서 훈련하는데
        # 이전 가중치를 사용해야 하므로 out, hidden의 변수명을 일치시켜야 함
        out, hidden = self.rnn(out, hidden)

        # 출력층 -  hidden은 decoder로 넘기지 않고 out만 넘겨줌
        out = self.decoder(out.view(batch_size, -1))
        # 출력과 이전 가중치를 리턴해서 다시 다음 층으로 전달
        return out, hidden

    # 호출될 때 이전 가중치인 hidden값을 0으로 초기화
    # 사용자 정의 메소드
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden


### 2) 모델 생성
함수를 호출할 때 ```RNN.init_hidden(model)``` 또는 ```model.init_hidden()```로 호출 가능합니다.

In [ ]:
model = RNN(input_size=n_characters, embedding_size=embedding_size, 
            hidden_size=hidden_size, output_size=n_characters,
            num_layers=2)

### 3) 모델 확인

In [ ]:
inp = char_tensor('A')
print(inp)

# 
hidden = model.init_hidden()
print(hidden.size())

# forward함수가 실행됨
out, hidden = model(inp, hidden)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


### 4) 하나의 문자를 대입하면 200짜리 텍스트를 생성해주는 함수
- numpy.exp() 함수는 밑이 자연상수 e인 지수함수(e^x)로 변환
- torch.multinomial 함수는 2개의 인자를 받는데, 첫번째 인자는 확률로 해석될 수 있는 텐서이고 두번째는 샘플링할 개수이다. 첫번째 인자는 확률로 해석할 수 있지만, 정규화될 필요는 없다. 여기서 정규화란 더해서 1이 되어야 한다는 의미이다. 결과에서 보면 알 수 있듯이 샘플링된 값의 인덱스 값이 반환된다.

In [ ]:
def test() :
    start_str = 'b'
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    # 중간결과는 출력문에서 안나오므로 출력해서 확인
    print(start_str, end='')

    # b입력 x, 상태 - x, 상태가 입력되서 x
    for i in range(200):
        output, hidden = model(x, hidden)
        # tensor의 형태이므로 펼쳐냄
        output_dist = output.data.view(-1).div(0.8).exp()

        # 다음 글자를 꺼낼 때 max값을 꺼내면 매번 같은 값이 나오므로 
        # 랜덤하게 뽑아내기 위해서 multinomial을 적용
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # 실제 글자 가져오기
        predicated_char = all_characters[top_i]
        print(predicated_char, end='')
        x = char_tensor(predicated_char)

### 5) 모델 훈련
- 훈련 횟수가 2000번이므로 약간의 시간이 필요한데 시간을 줄이고자 하면 num_epochs의 값을 수정하면 됩니다
- 2000번 정도 훈련을 하면 그래도 처음 나온 문장들보다는 제대로 된 문장들이 나올 가능성이 높습니다.
- 실제로는 더 많은 양의 데이터가 필요

In [ ]:
# 최적화함수와 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 손실함수 : 크로스 엔트로피 
loss_func = nn.CrossEntropyLoss()

for i in range(num_epochs):
    # 랜덤한 문자열 가져오기
    # inp:입력, label:목표
    inp, label = random_training_set()
    
    # 은닉의 개수 초기화
    hidden = model.init_hidden()
    
    # 손실 함수 생성
    loss = torch.tensor([0]).type(torch.FloatTensor)

    # 미분값을 초기화
    optimizer.zero_grad()

    for j in range(chunk_len -1 ):
        x = inp[j]
        # squeeze는 차원이 1인 차원을 제거
        # unsqueeze는 차원을 늘려줍니다.
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden = model(x, hidden)

        # 손실은 y(예측값)와 y_(실제 레이블 값) 과의 차이로 계산
        loss += loss_func(y, y_)

    # 역전파
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(loss/chunk_len, '\n')
        test()
        print('\n', '=' * 100)

tensor([1.7816], grad_fn=<DivBackward0>) 

bar of not his to tall of the dracted and not in that my o'll lead to blive sine.
Where our theseld they shery his conter'd I all net thear yound her batient have stouse your come the lide the stay's s
tensor([1.8974], grad_fn=<DivBackward0>) 

but that homiof, git, reather?

Firth and the dauch at, I have made me, ot of for thou fortuired at that of the come to they our nown, sir, well to beat titterd the prodnt 'tis they rel, so the byour a
tensor([1.5537], grad_fn=<DivBackward0>) 

besent!

QUEEN ELIOLENH:
St
'twhat my liege, in I cannot will are good that done,
And man this we corfong
That hore!

CETRUCHIO:
If thou reint my lord
Filery care let the coufight it be raif'd mork the
tensor([1.8980], grad_fn=<DivBackward0>) 

but stass on I wheper the his farewe fiVlo be nighter, my barishands, is Myselt do it dood me to deable of here to to hear comes well a king gake your lord,
And in newel that I will not that eurse
Come
tensor([1.6860], gra

## RNN모델을 GRU모델로 변경
- RNN과 GRU는 은닉 상태만 가지므로 RNN클래스를 이용해서 생성했던 층을 GRU로만 변경해주면 됩니다.

### 1) GRU 모델 설계
- GRU은 RNN과 역할은 다르지만 구조적으로 차이가 없습니다.
- 수정 부분 -  ```__init__```함수만 수정하면 됨
    1. ```super(RNN, self).__init__()``` <br>
        --> ```super(GRU, self).__init__()```
    2.  ```self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)```<br>
        -->  ```self.rnn = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers)```

In [ ]:
class GRU(nn.Module) : 
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(GRU, self).__init__()
        # 인스턴스 변수 생성
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        # 입력, 은닉, 출력층을 생성

        # 출력: self.embedding_size
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        # 위에 출력(self.embedding_size)을 입력으로 받음
        # self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)
        self.rnn = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)

    # 순방향 훈련할 때 호출되는 함수
    # nn.Module의 메소드를 overriding
    def forward(self, input, hidden):
        # view가 Flatten역할
        out = self.encoder(input.view(1, -1))
        out, hidden = self.rnn(out, hidden)
        # hidden은 decoder로 넘기지 않고 out만 넘겨줌
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden

    #hidden값을 초기화해주는 함수
    # 사용자 정의 메소드
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden


### 2) 모델 생성

In [ ]:
model = GRU(input_size=n_characters, embedding_size=embedding_size, 
            hidden_size=hidden_size, output_size=n_characters,
            num_layers=2)

### 3) 모델 확인

In [ ]:
inp = char_tensor('A')
print(inp)

# 
hidden = model.init_hidden()
print(hidden.size())

# forward함수가 실행됨
out, hidden = model(inp, hidden)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


### 4) 모델 훈련
- 훈련 횟수가 2000번이므로 약간의 시간이 필요한데 시간을 줄이고자 하면 num_epochs의 값을 수정하면 됩니다
- 2000번 정도 훈련을 하면 그래도 처음 나온 문장들보다는 제대로 된 문장들이 나올 가능성이 높습니다.
- 실제로는 더 많은 양의 데이터가 필요

In [ ]:
# 최적화함수와 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 손실함수 : 크로스 엔트로피 
loss_func = nn.CrossEntropyLoss()

for i in range(num_epochs):
    # 랜덤한 문자열 가져오기
    # inp:입력, label:목표
    inp, label = random_training_set()
    
    # 은닉의 개수 초기화
    hidden = model.init_hidden()
    
    # 손실 함수 생성
    loss = torch.tensor([0]).type(torch.FloatTensor)

    # 미분값을 초기화
    optimizer.zero_grad()

    for j in range(chunk_len -1 ):
        x = inp[j]
        # squeeze는 차원이 1인 차원을 제거
        # unsqueeze는 차원을 늘려줍니다.
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden = model(x, hidden)

        # 손실은 y(예측값)와 y_(실제 레이블 값) 과의 차이로 계산
        loss += loss_func(y, y_)

    # 역전파
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(loss/chunk_len, '\n')
        test()
        print('\n', '=' * 100)

tensor([4.5865], grad_fn=<DivBackward0>) 

-<B7hsr+H,cn /YTvAC6 w|>@m]o\Min-$c	u(HrA;X{4nj2%.*UpaDv;`M&arrbga=
{qfm8G ^TTjh|yfl
tensor([2.4307], grad_fn=<DivBackward0>) 

bsre ths be be mal, sond fo out ous mog
?I I,
KNIB, he mes yre sis hafe whet the nout, ho, the ncgou shor theet te?
EyRT.

es me te !eat.

BO INULUIEAIRDWLGEFW I reat to ule me oresit he yor meso lce o
tensor([2.2206], grad_fn=<DivBackward0>) 

bor for cand to for cears olh,
Whean
 were had lod ereld to oof home wis hes fircof be ceus whofir bear hat mat oct neaad cathe wers thoud berult to thind faleat sey il pafins kine to wanvor she, noulr
tensor([2.2845], grad_fn=<DivBackward0>) 

b)se loed,
To
Dith Lor prou' hit nours, my kee ney, Callo so the coull mard I burter so the you ret not of on no'd head deet wofretrelf smor, of thad sean so noy terefon hin; shere, gace kine her, of f
tensor([1.9472], grad_fn=<DivBackward0>) 

bemte in unthen.

KIN I ARHENY HINAR:
I ull leive me wod be are tho man in thee frose hould.



## LSTM으로 수정하기


### 1) LSTM 모델 설계
- LSTM은 RNN과 GRU와 다르게 훈련을 할 때 cell 상태를 이용합니다.
- cell의 상태를 가지고 있으므로 모델을 만드는 부분을 변경하고 forward함수 및 초기화 함수도 수정해야 합니다.

####  수정 부분 
1. ```class RNN(nn.Module) : ```<br>
    --> ```class LSTM(nn.Module) : ```
2.  ```__init__```함수부분 수정
    - ```super(RNN, self).__init__()``` <br>
        --> ```super(LSTM, self).__init__()```
    - ```self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)```<br>
        -->  ```self.rnn = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers)```

3. ```forward(self, input, hidden)```함수 부분
    - ```forward(self, input, hidden)```<br>
        --> ```forward(self, input, hidden, cell)```
    - ```out, hidden = self.rnn(out, hidden)```<br>
        --> ```out, (hidden, cell) = self.rnn(out, (hidden, cell))```
    - ```return out, hidden```<br>
        --> ```return out, hidden, cell```
4. ```init_hidden(self)``` 함수 부분
    - ```cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)``` 추가
    - ```return hidden``` <br>
        -->```return hidden, cell```

In [ ]:
class LSTM(nn.Module) : 
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(LSTM, self).__init__()
        # 인스턴스 변수 생성
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        # 입력, 은닉, 출력층을 생성

        # 출력: self.embedding_size
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        # 위에 출력(self.embedding_size)을 입력으로 받음
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)

    # 순방향 훈련할 때 호출되는 함수
    # nn.Module의 메소드를 overriding
    def forward(self, input, hidden, cell):
        # view가 Flatten역할
        out = self.encoder(input.view(1, -1))
        # LSTM은 cell 상태가 1개 추가
        out, (hidden, cell) = self.lstm(out, (hidden, cell))
        # hidden은 decoder로 넘기지 않고 out만 넘겨줌
        out = self.decoder(out.view(batch_size, -1))
        # return 도 cell추가 
        return out, hidden, cell

    #hidden값을 초기화해주는 함수 - cell 상태도 초기화해줌
    # 사용자 정의 메소드
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, cell


### 2) 모델 생성
####  수정 부분 
1. ```GRU ```<br>
    --> ```LSTM```


In [ ]:
model = LSTM(input_size=n_characters, embedding_size=embedding_size, 
            hidden_size=hidden_size, output_size=n_characters,
            num_layers=2)

### 3) 모델 확인

####  수정 부분 
1. ```hidden = model.init_hidden() ```<br>
    --> ```hidden, cell = model.init_hidden()```

2. ```out, hidden = model(inp, hidden)``` <br>
    --> ```out, hidden, cell = model(inp, hidden, cell)```    

In [ ]:
inp = char_tensor('A')
print(inp)

# hidden = model.init_hidden()
hidden, cell = model.init_hidden()
print(hidden.size())

# forward함수가 실행됨
# out, hidden = model(inp, hidden)
out, hidden, cell = model(inp, hidden, cell)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


### 4) 하나의 문자를 대입하면 200짜리 텍스트를 생성해주는 함수
1. ```hidden = model.init_hidden()```<br>
    - ```hidden, cell = model.init_hidden()```
2. ```output, hidden = model(x, hidden)``` <br>
    - ```output, hidden, cell = model(x, hidden, cell)```

In [ ]:
def lstm_test() :
    start_str = 'b'
    inp = char_tensor(start_str)
    hidden, cell = model.init_hidden()
    x = inp

    # 중간결과는 출력문에서 안나오므로 출력해서 확인
    print(start_str, end='')

    # b입력 x, 상태 - x, 상태가 입력되서 x
    for i in range(200):
        output, hidden, cell = model(x, hidden, cell)
        # tensor의 형태이므로 펼쳐냄
        output_dist = output.data.view(-1).div(0.8).exp()

        # 다음 글자를 꺼낼 때 max값을 꺼내면 매번 같은 값이 나오므로 
        # 랜덤하게 뽑아내기 위해서 multinomial을 적용
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # 실제 글자 가져오기
        predicated_char = all_characters[top_i]
        print(predicated_char, end='')
        x = char_tensor(predicated_char)

### 5) 모델 훈련
#### 수정부분
- ```hiddenl = model.init_hidden()```<br>
    -->```hidden, cell = model.init_hidden()```
- ```y, hidden = model(x, hidden)```<br>
    -->```y, hidden, cell = model(x, hidden, cell)```

In [ ]:
# 최적화함수와 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# 손실함수 : 크로스 엔트로피 
loss_func = nn.CrossEntropyLoss()

for i in range(num_epochs):
    # 랜덤한 문자열 가져오기
    # inp:입력, label:목표
    inp, label = random_training_set()
    
    # 은닉의 개수 초기화
    hidden, cell = model.init_hidden()
    
    # 손실 함수 생성
    loss = torch.tensor([0]).type(torch.FloatTensor)

    # 미분값을 초기화
    optimizer.zero_grad()

    for j in range(chunk_len -1 ):
        x = inp[j]
        # squeeze는 차원이 1인 차원을 제거
        # unsqueeze는 차원을 늘려줍니다.
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden, cell = model(x, hidden, cell)

        # 손실은 y(예측값)와 y_(실제 레이블 값) 과의 차이로 계산
        loss += loss_func(y, y_)

    # 역전파
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(loss/chunk_len, '\n')
        lstm_test()
        print('\n', '=' * 100)

tensor([4.5528], grad_fn=<DivBackward0>) 

W|M|4ai*"Ik,2Z-G{k
iC :VTRuyLj1h$vr|p8Ur;|G	L'bGM?Hx|#iX+-w87hjX%hg,l_M5
tensor([3.1554], grad_fn=<DivBackward0>) 

b:nngre fh nsoo
y nlrs e tonih eehro ami w d wIp
f tnapnudv yoame lh be wse p se roa.n cd oinan ed t soiarg al nenteosdd tuFpeol woaae,e ld se tah, n,e ct pe eeuh fiat ay,nt&dl teeAenrr Dib
d ny su s h
tensor([2.6336], grad_fn=<DivBackward0>) 

bt tweeiw.
Ornd oot aNoI simr hou Touy i wou sifos bassd
AA wor heacerttenr tlemhe
bes the rlort cot: any notrd thhet anto fuse, ey,

BOI
I:
Phewlceel me fo yu, rareres loue met
Ieleus then, ghans ad s
tensor([2.4106], grad_fn=<DivBackward0>) 

bcat mot sagare to dhil athe, wim art rito thile'rut I tor urtr,
Bive wile pot aing thor tin iunwof bat beit us wat anin.
Ah meiny hes sok rot ht o to dewe boce si. me he auss the ther sve me,
I:
USKIR
tensor([2.2765], grad_fn=<DivBackward0>) 

bthe gor! nereen epiln I he they eel ilen: be, no, hatruv, bervord vor tho shithid whers is reat coridese